In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display,clear_output
import pandas as pd
import matplotlib.pyplot as plt
import datajoint as dj
from pipeline import pipeline_tools, lab, experiment, behavior_foraging
import numpy as np
dj.conn()
from foraging_inspector_utils import *
#%matplotlib widgetsnbextension

Connecting rozmar@mesoscale-activity.datajoint.io:3306


In [2]:


def update_params(change=None):  # this script make sure that the widgets are synchronized
    plottype = task_selector.value
    if '2' in plottype:
        wr_names = (behavior_foraging.SessionTaskProtocol()*experiment.Session()*lab.WaterRestriction()&'session_task_protocol = 100').fetch('water_restriction_number')
    elif '3' in plottype:
        wr_names = (behavior_foraging.SessionTaskProtocol()*experiment.Session()*lab.WaterRestriction()&'session_task_protocol = 101').fetch('water_restriction_number')
    wr_names = np.unique(wr_names)
    wr_name_selector.options = wr_names
    wr_name = wr_name_selector.value
    subject_id = (lab.WaterRestriction()&'water_restriction_number = "{}"'.format(wr_name)).fetch1('subject_id')

    df_sessions = pd.DataFrame(behavior_foraging.SessionTaskProtocol()*experiment.Session()&'subject_id = {}'.format(subject_id))
    if '2' in plottype:
        needed = df_sessions['session_task_protocol'] == 100
    elif '3' in plottype:
        needed = df_sessions['session_task_protocol'] == 101
    df_sessions=df_sessions[needed]
    if len(df_sessions)>0:
        try:
            session_selector.min = df_sessions['session'].min()
            session_selector.max = df_sessions['session'].max()
        except:
            session_selector.max = df_sessions['session'].max()
            session_selector.min = df_sessions['session'].min()
    else:
        session_selector.min = 0
        session_selector.max = 0
    session_real_foraging = (behavior_foraging.SessionTaskProtocol()&'subject_id = {}'.format(subject_id)).fetch('session_real_foraging')
    metadata_display.value = 'Pretraining session num: {}'.format(sum(session_real_foraging ==0))

def plot_stuff(dummy=None): # this script does the plotting
    # generate figure structure
    fig=plt.figure()
    ax1=fig.add_axes([0,0,2,.8])
    ax2=fig.add_axes([0,-.6,2,.4])
    ax3=fig.add_axes([0,-1.6,2,.8])
    ax4=fig.add_axes([0,-2.6,2,.8])
    ax5 = fig.add_axes([0,-3.6,2,.8])
    plt.close(fig)
    
    with output:
        clear_output(wait=True)
        print('waiting for data..')
    # invoke plot functions    
    plot_trials(ax1,
                ax2,
                plottype = task_selector.value,
                wr_name = wr_name_selector.value,
                sessions = session_selector.value,
                plot_every_choice= plot_each_choice.value,
                show_bias_check_trials =  bias_check_selector.value,
                choice_filter = np.ones(plot_choice_averaging_window.value)/plot_choice_averaging_window.value)

    plot_efficiency_matching_bias(ax3,
                                  plottype = task_selector.value,
                                  wr_name = wr_name_selector.value,
                                  sessions= session_selector.value,
                                  plot_efficiency_type = plot_efficiency.value)
    plot_rt_iti(ax4,
                ax5,
                plottype = task_selector.value,
                wr_name = wr_name_selector.value,
                sessions = session_selector.value,
                show_bias_check_trials =  bias_check_selector.value,
                kernel = np.ones(plot_choice_averaging_window.value)/plot_choice_averaging_window.value)
    # display figure on widget
    with output:
        clear_output(wait=True)
        display(fig) 
        

        
## widgets are defined here
border = widgets.Output(layout={'border': '1px solid black'})
output = widgets.Output()
output2 = widgets.Output()
task_selector = widgets.Dropdown(layout={'width': 'auto'},
                                    options=['2lickport','3lickport'],
                                    disabled=False)
task_selector.observe(update_params,'value')

wr_name_selector = widgets.Dropdown(layout={'width': 'auto'},
                                    options=[],
                                    disabled=False)
wr_name_selector.observe(update_params,'value') # runs update params upon change in value

session_selector = widgets.IntRangeSlider(value=[5, 7],
                                          min=0,
                                          max=10,
                                          step=1,
                                          disabled=False,
                                          continuous_update=False,
                                          orientation='horizontal',
                                          readout=True,
                                          readout_format='d')

bias_check_selector = widgets.Checkbox(value=True,
                                       description='Include bias check',
                                       disabled=False,
                                       indent=False,
                                       layout={'width': 'auto'})
metadata_display = widgets.HTML(value="",
                                placeholder=' ',
                                description=' ',
                                layout={'width': 'auto'})



plot_button = widgets.Button(description="plot")
plot_button.on_click(plot_stuff)

# plotting related widgets
plot_each_choice = widgets.Checkbox(value=True,
                              description='Plot each choice',
                              disabled=False,
                              indent=False,
                              layout={'width': 'auto'})
plot_choice_averaging_window = widgets.IntText(value=10,
                                               description='Choice avg window:',
                                               style={'description_width': 'initial'},
                                               disabled=False,
                                               layout={'width': '70%'}
                                              )
plot_efficiency = widgets.Dropdown(layout={'width': 'initial'},
                                   options=['max_prob','sum_prob','max_available','sum_available'],
                                   value = 'sum_prob',
                                   description = 'Efficiency:',
                                   disabled=False)


## layout of widgets
controls = widgets.HBox([widgets.VBox([widgets.Label('Task'), task_selector]),
                         widgets.VBox([widgets.Label('Subject'), wr_name_selector]),
                         widgets.VBox([widgets.Label('Sessions'),session_selector]),
                         widgets.VBox([bias_check_selector]),
                         widgets.VBox([plot_button])])

plot_controls = widgets.HBox([widgets.VBox([plot_each_choice,plot_choice_averaging_window]),
                             widgets.VBox([plot_efficiency])])
accordion = widgets.VBox(children=(controls,plot_controls,metadata_display), title=('Select data', 'Plot details','Metadata'))
#accordion.set_title(0, 'Data')
#accordion.set_title(1, 'Plot')


# displaying widgets
display(accordion)
display(border)
display(widgets.VBox([output,output2]))
display(border)
update_params()

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))